In [40]:
import sys
import os
sys.path.append(os.getcwd())
from src.taxodist import td_utils as utils
from src.taxodist import tree_parsers
from src.taxodist import td_calc
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pylab as plt

tree = tree_parsers.getICD10GMTree(version='2021')
td = td_calc.Taxodist()
#plt.style.use("seaborn")

In [11]:
df_hd_moltb = pd.read_excel('analysis/resources/pseudo_icd10_hauptdiagnose_moltb.xlsx')
df_pancreas_hd = df_hd_moltb[df_hd_moltb['ICD-10 Hauptdiagnose'].str.contains('C25')]

df_moltb = pd.read_excel('analysis/resources/pseudo_icd10_moltb.xlsx')
df_patient_icd = df_moltb.groupby(df_moltb.PseudoPatNr.name)['ICD'].agg(set).reset_index(name='ICD')
df_pancreas_patient = df_patient_icd[df_patient_icd['PseudoPatNr'].isin(df_pancreas_hd['Pseudonym'])]

patient_diagnoses = df_pancreas_patient['ICD'].to_numpy()
patient_diagnoses = [str(diag_set) for diag_set in patient_diagnoses]

pancreas_patient_array = df_pancreas_patient.to_numpy()
pancreas_icd_sets = [set(icd_list) for patient, icd_list in pancreas_patient_array]

In [ ]:
# use taxodist to calculate similarity of all patients
dist_matrix = td.calc_set_sim(pancreas_icd_sets,tree,'levels','nguyen_almubaid','bipartite_matching',normalize=False)

df_mds_coordinates = utils.getMDSMatrix(dist_matrix)

In [38]:
#dist_matrix = pd.read_excel('analysis/generated/matrices/pancreas_pats_dist_L_NA_BIP.xlsx')
dist_matrix = dist_matrix.to_numpy()
setnames = ['patient ' + str(count) for count in range(0,len(dist_matrix))]
df_mds_coordinates.set_axis(setnames,axis='index',inplace=True)
df_mds_coordinates['Patient'] = setnames

df_mds_coordinates['ICD diagnoses'] = patient_diagnoses
df_mds_coordinates.rename(columns={0:'X',1:'Y'}, inplace=True)
# utils.plotDistMatrix(df_mds_coordinates,setnames)

fig = px.scatter(df_mds_coordinates,x='X',y='Y',hover_data={'Patient':True,'ICD diagnoses':True,'X':False,'Y':False})
fig.show()

In [39]:
df_pancreas_hd

,ICD-10 Hauptdiagnose,Pseudonym
1,C25.1,c856aac7ae6ea8502e45368749f9d82c
8,C25.0,8c492854a1494a616ee0a1f38f3ac971
10,C25.9,24a2aab4b0f45dfc0daa987b267927a8
13,C25.0,01a7076d03600a3d69cc43e181bc4bc0
16,C25.0,241852e144299cb99c6a56457b77a4e9
18,C25.2,2192148f8783d0a2b02e48260528f27d
19,C25.2,2192148f8783d0a2b02e48260528f27d
21,C25.0,1df425d9aad0848a8cea3aed68cfeebe
25,C25.0,b74fd3befe21940e6c1e838a5b525091
26,C25.2,348ffd6b9debc52ee43742f6c0d625b4


In [ ]:
tree = tree_parsers.getICD10GMTree()
depth = tree.depth()
td = td_calc.Taxodist()

set1 = {'C00.0','C02.0','C04.0','C05.0','C06.0'}
set2 = {'D00.0','C02.0','C04.0','C05.0','C06.0'}
set3 = {'D00.0','D02.0','C04.0','C05.0','C06.0'}
set4 = {'D00.0','D02.0','D04.0','C05.0','C06.0'}
set5 = {'D00.0','D02.0','D04.0','D05.0','C06.0'}

# sets = [utils.getRandomConcepts(10,tree) for x in range(0,10)]
sets = [set1,set2,set3,set4,set5]
column_names=['Hauptdiagnose','ND1','ND2','ND3','ND4',]

setnames = ['patient ' + str(x) for x in range(1,6)]

In [ ]:
dist_matrix = td.calc_set_sim(sets,tree,'levels','nguyen_almubaid','bipartite_matching')
sim_matrix = 1.0 - dist_matrix.round(3)
df_mds_coordinates = utils.getMDSMatrix(dist_matrix)

# fig = px.sunburst(
# sets,
# names='character',
# parents='parent',
# values='value',
# )
# fig.show()


utils.plotDistMatrix(df_mds_coordinates,setnames)
dist_matrix

In [ ]:
dist_matrix = td.calc_set_sim(sets,tree,'levels','nguyen_almubaid','bipartite_matching',normalize=False)
sim_matrix = 1.0 - dist_matrix.round(3)
df_mds_coordinates = utils.getMDSMatrix(dist_matrix)

# fig = px.sunburst(
# sets,
# names='character',
# parents='parent',
# values='value',
# )
# fig.show()


utils.plotDistMatrix(df_mds_coordinates,setnames)

In [ ]:
df_sets = pd.DataFrame([['C00.0','C02.0','C04.0','C05.0','C06.0'],['D00.0','C02.0','C04.0','C05.0','C06.0'],['D00.0','D02.0','C04.0','C05.0',
            'C06.0'],['D00.0','D02.0','D04.0','C05.0','C06.0'],['D00.0','D02.0','D04.0','D05.0','C06.0']],index=setnames,columns=column_names)

df_hd = df_sets.Hauptdiagnose.unique()
nd = [df_sets[c].unique() for c in df_sets.columns[1:5]]
nd_flat = flat_list = [item for sublist in nd for item in sublist]

#ct = pd.crosstab(index=df_sets.Hauptdiagnose,columns=df_sets.columns,ag)
#ct